Type checking via a decorator.

In [1]:
import numpy as np
import pandas as pd
from type_signature import non_null_types_in_frame


In [2]:
import import_tc

We add a decorator that shows the types of at least a subset of positional and named arguments. Declarations are either Python types, or sets of types. A special case is Pandas data frames, where we specify a required subset of columns and their value type-sets. "return_spec" is reserved to name the return type of the function (so the function we are working with may not have an argument of that name).

In [3]:
@import_tc.TypeSignature({
        'a': int, 
        'b': int, 
        'c': {'x': int},
        },
        return_spec={'z': float})
def fn(a, /, b, *, c, d=None):
    """doc"""
    return d

In [4]:
help(fn)

Help on function fn in module __main__:

fn(a, /, b, *, c, d=None)
     arg specifications
    {'a': <class 'int'>, 'b': <class 'int'>, 'c': {'x': <class 'int'>}}
     return specification:
    {'z': <class 'float'>}
    
    
    doc



In [5]:
try:
    fn()
except TypeError as e:
    print(e)


function fn(), issues:
expected arg a missing  
expected arg b missing  
expected arg c missing


In [6]:
try:
    fn(1, 2)
except TypeError as e:
    print(e)


function fn(), issues:
expected arg c missing


In [7]:
try:
    fn(1, 2, c=3)
except TypeError as e:
    print(e)


function fn(), issues:
arg c expected type pandas.DataFrame, found type int


In [8]:
try:
    fn(1, 2, c=pd.DataFrame({'z': [7]}))
except TypeError as e:
    print(e)


function fn(), issues:
arg c missing required column 'x'


In [9]:
fn(1, 2, c=pd.DataFrame({'x': [3]}), d=pd.DataFrame({'z': [7.0]}))

,z
0,7.0


In [10]:
fn(1, b=2, c=pd.DataFrame({'x': [3]}), d=pd.DataFrame({'z': [7.0]}))

,z
0,7.0


In [11]:
try:
    fn(1, 2, c=pd.DataFrame({'x': [3.0]}))
except TypeError as e:
    print(e)


function fn(), issues:
arg c  column 'x' expected type int, found type float


In [12]:
rv = None
try:
    fn(1, 2, c=pd.DataFrame({'x': [30], "z": [17.2]}), d=pd.DataFrame({'q': [7.0]}))
except TypeError as e:
    print(e.args[0])
    rv = e.args[1]

rv


fn() return value: missing required column 'z'


,q
0,7.0


In [13]:
@import_tc.TypeSignature(
        {'a': pd.DataFrame},
        return_spec=int,
)
def g(a):
    return a.shape[0]

In [14]:
try:
    g(a=7)
except TypeError as e:
    print(e)


function g(), issues:
arg a expected type DataFrame, found type int


In [15]:
try:
    g(7)
except TypeError as e:
    print(e)


function g(), issues:
arg a expected type DataFrame, found type int


In [16]:
g(a=pd.DataFrame({'x': [5]}))

1

In [17]:
try:
    g({'x': [5]})
except TypeError as e:
    print(e)


function g(), issues:
arg a expected type DataFrame, found type dict


In [18]:
d =  pd.DataFrame({
        'b': [1, 3, 4],
        'q': np.nan,
        'r': [1, None, 3],
        's': [np.nan, 2.0, 3.0],
        'x': [1, 7.0, 2],
        'y': ["a", None, np.nan],
        'z': [1, 1.0, False],
    })

d

,b,q,r,s,x,y,z
0,1,NaN,1.0,NaN,1.0,a,1
1,3,NaN,NaN,2.0,7.0,None,1.0
2,4,NaN,3.0,3.0,2.0,NaN,False


In [19]:
d.dtypes

b      int64
q    float64
r    float64
s    float64
x    float64
y     object
z     object
dtype: object

In [20]:
non_null_types_in_frame(d)

{'b': {numpy.int64},
 'q': None,
 'r': {numpy.float64},
 's': {numpy.float64},
 'x': {numpy.float64},
 'y': {str},
 'z': {bool, float, int}}